In [ ]:
%pip install  git+https://github.com/CompVis/taming-transformers.git
!git clone https://github.com/CompVis/taming-transformers.git && cd taming-transformers && python -m pip install -e .
%pip install "omegaconf==2.0.0" "pytorch-lightning==1.0.8" einops transformers
%pip install -q git+https://github.com/cthiounn/dalle-tiny.git
from tqdm import tqdm
import s3fs
import os

# Create filesystem object
S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
fs = s3fs.S3FileSystem(client_kwargs={'endpoint_url': S3_ENDPOINT_URL})
BUCKET = "cthiounn2"
fs.ls(BUCKET)

files=['checkpoint_reverse_pokemon_launch2022_05_28_10248.pth','model_vqgan_minidalle.ckpt','config_vqgan_minidalle.yaml']
for file in tqdm(files):
    with fs.open(f'{BUCKET}/{file}', mode="rb") as file_in, open(file,"wb") as file_out:
            file_out.write(file_in.read())

import os
os._exit(00)

In [ ]:
import sys
sys.path.append(".")

# also disable grad to save memory
from omegaconf import OmegaConf
import taming
from taming.models.vqgan import VQModel
import torch
torch.set_grad_enabled(False)

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")



config_path = "config_vqgan_minidalle.yaml"
config = OmegaConf.load(config_path)
vqmodel=VQModel(**config.model.params).to(DEVICE)

ckpt_path = "model_vqgan_minidalle.ckpt"

sd = torch.load(ckpt_path, map_location=DEVICE)["state_dict"]
vqmodel.load_state_dict(sd, strict=False)

In [ ]:
from dalle_tiny.model import TinyDalleModel
import torch
import torch.optim as optim
import torch.nn.functional as F
import torch.nn as nn
from torch.utils.data import DataLoader

device = 'cuda' if torch.cuda.is_available() else 'cpu'

from transformers import BartForConditionalGeneration

try:
    model=TinyDalleModel.from_pretrained('.')
except:
    model=TinyDalleModel.from_pretrained('facebook/bart-large-cnn')
model.reinit_model_for_images()

In [ ]:
captions=[]
captions.append("Pikachu et Dragofeu")
captions.append("Pikachu")
captions.append("Bulb i zarre")
captions.append("Bulb")

In [ ]:
import matplotlib.pyplot as plt
from transformers import BartTokenizer
from collections import defaultdict
import imageio
import numpy as np
import torchvision.transforms as T
from PIL import Image
CUDA_LAUNCH_BLOCKING=1
tokenizer=BartTokenizer.from_pretrained('facebook/bart-large-cnn')

image_dict=defaultdict(list)
# file_in="checkpoint_test_2022_05_02_191000.pth"
file_in="checkpoint_reverse_pokemon_launch2022_05_28_10248.pth"

model.load_state_dict(torch.load(file_in,map_location=device))
model.config.eos_token_id=16384
model.config.max_length=257
model.eval()
model=model.to(device)

num_ret=10

images=[]
for j,caption in enumerate(captions):
  inputs=tokenizer(caption, return_tensors="pt",max_length=256,padding="max_length")
  inputs=inputs.to(device)
  pred=model.generate(**inputs,
                      # do_sample=False,
                      # num_beams=10,
                      do_sample=True,
                      top_p=0.90,
                      temperature=1.7,
                      top_k=300,
                      # top_k=25,

                      num_return_sequences=num_ret
                      )
  pred=pred.detach()
  pred=pred.squeeze()
  
  output_indices=torch.Tensor(num_ret,256)
  output_indices[:]=0
  output_indices[:,:256]=pred[:,1:]
  output_indices=output_indices.to(torch.long)
  output_indices.to("cpu")
  vqmodel=vqmodel.to("cpu")


  z_q = vqmodel.quantize.embedding(output_indices).reshape(num_ret, 16, 16, 256).permute(0,3,1,2)
  u=vqmodel.decode(z_q).add(1).div(2).cpu()
  image_dict[j].append(u)
  video_file=caption.replace('.','').replace(' ','_').lower()
  for i in range(num_ret):
    img=u[i,:,:,:].cpu().squeeze().permute(1, 2, 0)
    img.squeeze().clip(0.0, 1.0)
    
    ii=np.asarray(img * 255, dtype=np.uint8)


    aa=Image.fromarray(ii)
    images.append(ii)
    aa.save(video_file+str(i)+".jpg")

In [ ]:
def image_grid(array, ncols=5):
    index, height, width, channels = array.shape
    nrows = index//ncols
    img_grid = (array.reshape(nrows, ncols, height, width, channels)
              .swapaxes(1,2)
              .reshape(height*nrows, width*ncols,channels))
    
    return img_grid

result = image_grid(np.array(images))
fig = plt.figure(figsize=(40., 40.))
plt.imshow(result)

In [ ]:
! rm -rf *.jpg